In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [2]:
data = pd.read_csv('PEP.csv')


In [3]:
data

,ticker,date,open,high,low,close
0,PEP,1972-06-01,0.000,1.60,1.580,1.60
1,PEP,1972-06-02,1.600,1.62,1.600,1.60
2,PEP,1972-06-05,1.600,1.60,1.540,1.57
3,PEP,1972-06-06,1.560,1.56,1.560,1.56
4,PEP,1972-06-07,1.560,1.56,1.550,1.55
...,...,...,...,...,...,...
12826,PEP,2023-04-13,182.620,184.61,182.105,184.38
12827,PEP,2023-04-14,183.335,184.02,182.920,183.51
12828,PEP,2023-04-17,184.190,184.75,183.481,184.45
12829,PEP,2023-04-18,184.770,185.22,183.740,184.83


In [4]:
data = data.filter(['close'])


In [5]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
11831,127.24
11832,126.83
11833,127.67
11834,127.27
11835,125.98
...,...
12826,184.38
12827,183.51
12828,184.45
12829,184.83


In [6]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [7]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [8]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [9]:
start_time = time.time()

In [10]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(3):
    model.add(Dense(units=100, activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [11]:
history = model.fit(X_train, y_train, epochs=11000, batch_size=8)

Epoch 1/11000
87/87 [==============================] - 2s 2ms/step - loss: 0.0565
Epoch 2/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0100
Epoch 3/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0068
Epoch 4/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0058
Epoch 5/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 6/11000
87/87 [==============================] - 0s 3ms/step - loss: 0.0052
Epoch 7/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 8/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0039
Epoch 9/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 10/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0032
Epoch 11/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 12/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 13/1100

87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 197/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.5030e-04
Epoch 198/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.9790e-04
Epoch 199/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.9203e-04
Epoch 200/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 201/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 202/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 203/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 204/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.6736e-04
Epoch 205/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.8724e-04
Epoch 206/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 207/11000
87/87 [==============================] - 0s 2ms/step - l

87/87 [==============================] - 0s 2ms/step - loss: 9.9656e-04
Epoch 387/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.9988e-04
Epoch 388/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 389/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.5496e-04
Epoch 390/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.6169e-04
Epoch 391/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 392/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.9658e-04
Epoch 393/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.9126e-04
Epoch 394/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.2313e-04
Epoch 395/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.3894e-04
Epoch 396/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 397/11000
87/87 [==============================] - 0s 

87/87 [==============================] - 0s 2ms/step - loss: 9.1290e-04
Epoch 575/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.1515e-04
Epoch 576/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.7805e-04
Epoch 577/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.2050e-04
Epoch 578/11000
87/87 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 579/11000
87/87 [==============================] - 0s 3ms/step - loss: 9.2508e-04
Epoch 580/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.5736e-04
Epoch 581/11000
87/87 [==============================] - 0s 3ms/step - loss: 9.7529e-04
Epoch 582/11000
87/87 [==============================] - 0s 3ms/step - loss: 9.4550e-04
Epoch 583/11000
87/87 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 584/11000
87/87 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 585/11000
87/87 [==============================] - 0s 

87/87 [==============================] - 0s 2ms/step - loss: 9.6520e-04
Epoch 763/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.2689e-04
Epoch 764/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.6619e-04
Epoch 765/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.1562e-04
Epoch 766/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.6499e-04
Epoch 767/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.9678e-04
Epoch 768/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.4562e-04
Epoch 769/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.5442e-04
Epoch 770/11000
87/87 [==============================] - 0s 2ms/step - loss: 8.7202e-04
Epoch 771/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.3629e-04
Epoch 772/11000
87/87 [==============================] - 0s 2ms/step - loss: 9.1394e-04
Epoch 773/11000
87/87 [=========================

...

87/87 [==============================] - 0s 3ms/step - loss: 5.9951e-04
Epoch 10147/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.7548e-04
Epoch 10148/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.8193e-04
Epoch 10149/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.5873e-04
Epoch 10150/11000
87/87 [==============================] - 0s 2ms/step - loss: 6.1288e-04
Epoch 10151/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.7764e-04
Epoch 10152/11000
87/87 [==============================] - 0s 2ms/step - loss: 5.9273e-04
Epoch 10153/11000
87/87 [==============================] - 0s 2ms/step - loss: 5.9596e-04
Epoch 10154/11000
87/87 [==============================] - 0s 2ms/step - loss: 6.5291e-04
Epoch 10155/11000
87/87 [==============================] - 0s 2ms/step - loss: 5.8837e-04
Epoch 10156/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.3915e-04
Epoch 10157/11000
87/87 [===

87/87 [==============================] - 0s 3ms/step - loss: 7.4910e-04
Epoch 10329/11000
87/87 [==============================] - 0s 2ms/step - loss: 5.4430e-04
Epoch 10330/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1684e-04
Epoch 10331/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1336e-04
Epoch 10332/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.8186e-04
Epoch 10333/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.6374e-04
Epoch 10334/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.6641e-04
Epoch 10335/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1983e-04
Epoch 10336/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.7633e-04
Epoch 10337/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.0358e-04
Epoch 10338/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.5674e-04
Epoch 10339/11000
87/87 [===

87/87 [==============================] - 0s 3ms/step - loss: 5.8312e-04
Epoch 10511/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.0450e-04
Epoch 10512/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1484e-04
Epoch 10513/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.9460e-04
Epoch 10514/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.7966e-04
Epoch 10515/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.4111e-04
Epoch 10516/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.2291e-04
Epoch 10517/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.4776e-04
Epoch 10518/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.4441e-04
Epoch 10519/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.9170e-04
Epoch 10520/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.3327e-04
Epoch 10521/11000
87/87 [===

87/87 [==============================] - 0s 3ms/step - loss: 5.8532e-04
Epoch 10693/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.2385e-04
Epoch 10694/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.4991e-04
Epoch 10695/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.5872e-04
Epoch 10696/11000
87/87 [==============================] - 0s 3ms/step - loss: 8.1640e-04
Epoch 10697/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1069e-04
Epoch 10698/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.3199e-04
Epoch 10699/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.1702e-04
Epoch 10700/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.6439e-04
Epoch 10701/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.8043e-04
Epoch 10702/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.8182e-04
Epoch 10703/11000
87/87 [===

87/87 [==============================] - 0s 3ms/step - loss: 7.0041e-04
Epoch 10875/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.8035e-04
Epoch 10876/11000
87/87 [==============================] - 0s 3ms/step - loss: 5.9991e-04
Epoch 10877/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.6364e-04
Epoch 10878/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.5037e-04
Epoch 10879/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.0728e-04
Epoch 10880/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.0558e-04
Epoch 10881/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.0836e-04
Epoch 10882/11000
87/87 [==============================] - 0s 3ms/step - loss: 7.3286e-04
Epoch 10883/11000
87/87 [==============================] - 0s 3ms/step - loss: 6.3023e-04
Epoch 10884/11000
87/87 [==============================] - 0s 3ms/step - loss: 9.6178e-04
Epoch 10885/11000
87/87 [===

In [12]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

10/10 [==============================] - 0s 3ms/step
Training time: 2424.1877377033234 seconds


In [13]:
import h5py
from keras.models import save_model

# Lưu mô hình CNN vào file model.h5
save_model(model, 'CNN_PEP_73.h5')

# Tạo file H5 mới
with h5py.File('CNN_PEP_73_data.h5', 'w') as f:
    # Lưu giá trị y_pred
    f.create_dataset('y_test', data=y_test)
    
    # Lưu giá trị X_test
    f.create_dataset('X_test', data=X_test)
    
    # Lưu giá trị training_time
    f.create_dataset('training_time', data=training_time)
